<a href="https://colab.research.google.com/github/cwinsor/uml_twitter/blob/main/colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"hello"

'hello'